## In this notebook, we will run through some preliminary data pre-processing in Python
We will with the equity_df variable, which is a dataframe of fundamental and price data for US equities from going back to 1995

In [1]:
import pandas as pd
import numpy as np
import datetime 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

equity_df = pd.read_csv("C:/Users/lbianculli/dev/us_equities/python_processes/all_fundamentals_macro2.csv").drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)
equity_df["date"] = equity_df["date_dt_x"].astype(str)
equity_df["date_dt"] = pd.to_datetime(equity_df["date"], format='%Y-%m-%d')

# some of our data is not filled in
equity_df[["average_assets", "average_equity", "invested_capital_average", "return_on_average_assets", 
                            "return_on_average_equity", "return_on_invested_capital", "return_on_sales"]].head()

,average_assets,average_equity,invested_capital_average,return_on_average_assets,return_on_average_equity,return_on_invested_capital,return_on_sales
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# drop these as they have bad data
equity_df = equity_df.drop(["average_assets", "average_equity", "invested_capital_average", "return_on_average_assets", 
                            "return_on_average_equity", "return_on_invested_capital", "return_on_sales"], axis=1)

## Setup additional Data points
Our data comes prepared with quite a few fundamental measures. We will supplement that by adding a few of our own, including basic calculations like leverage and roe.

In [3]:
# basic calcs
equity_df["leverage"] = equity_df["total_assets"] / equity_df["shareholders_equity"]
equity_df["buyback_yield"] = -equity_df["issuance_purchase_of_equity_shares"] / equity_df["market_capitalization"]
equity_df["total_yield"] = equity_df["buyback_yield"] + equity_df["dividend_yield"]
equity_df["roe"] = equity_df['earnings_before_interest_taxes_depreciation_amortization_ebitda'] / equity_df["shareholders_equity"].shift(1)
equity_df["roa"] = equity_df['earnings_before_interest_taxes_depreciation_amortization_ebitda'] / equity_df["total_assets"].shift(1)
equity_df["asset_turnover"] = equity_df["revenues_usd"] / equity_df["total_assets"]
equity_df["debt_to_assets"] = equity_df["total_debt"] / equity_df["total_assets"]

# value factors
equity_df = equity_df.replace(np.inf, np.nan)
equity_df = equity_df.replace(-np.inf, np.nan)
equity_df = equity_df.fillna(equity_df.mean())

# Other factors of interest
equity_df["intangibles_to_assets"] = equity_df["goodwill_and_intangible_assets"] / equity_df["total_assets"]
equity_df["rnd_to_revenue"] = equity_df["research_and_development_expense"] / equity_df["revenues"]
equity_df["rnd_to_revenue"] = equity_df["rnd_to_revenue"].replace(np.nan, 0.0)

# equity_df = equity_df.dropna()
equity_df.dropna().shape, equity_df.shape

((40403, 165), (42729, 165))

### Handling outliers

First we will look to remove outstanding outliers in the returns columns. To do this, we will use boundaries based on the IQR, which is the difference between Q3 and Q1. Then we are going to use an IsolationForest to locate and remove outliers in feature space. 

Link to the IsolationForest paper: https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf

In [4]:
# calculate first and third quantile for desired measures
log_ret_q1 = equity_df['1mo_log_rets'].quantile(0.25)
log_ret_q3 = equity_df['1mo_log_rets'].quantile(0.75)
fwd_log_ret_q1 = equity_df['1mo_fwd_log_rets'].quantile(0.25)
fwd_log_ret_q3 = equity_df['1mo_fwd_log_rets'].quantile(0.75)

# get IQR
log_ret_iqr = log_ret_q3 - log_ret_q1
fwd_log_ret_iqr = fwd_log_ret_q3 - fwd_log_ret_q1

# locate outliers based on 1.5 * IQR
equity_df["log_outlier"] = np.where((equity_df["1mo_log_rets"] < (log_ret_q1 - 1.5*log_ret_iqr)) | \
                                    (equity_df["1mo_log_rets"] > (log_ret_q3 + 1.5*log_ret_iqr)), 1, 0)

equity_df["fwd_log_outlier"] = np.where((equity_df["1mo_fwd_log_rets"] < (fwd_log_ret_q1 - 1.5*fwd_log_ret_iqr)) | \
                                    (equity_df["1mo_fwd_log_rets"] > (fwd_log_ret_q3 + 1.5*fwd_log_ret_iqr)), 1, 0)

# remove outliers
print(f"Removed {equity_df.shape[0] - equity_df[(equity_df['log_outlier'] == 0) & (equity_df['fwd_log_outlier'] == 0)].shape[0]} outliers.")
equity_df = equity_df[(equity_df["log_outlier"] == 0) & (equity_df["fwd_log_outlier"] == 0)]
equity_df["rnd_to_revenue"] = equity_df["rnd_to_revenue"].replace(np.nan, 0.0)

equity_df = equity_df.replace(np.inf, np.nan)
equity_df["rnd_to_revenue"] = equity_df["rnd_to_revenue"].replace(np.nan, 0.0)


Removed 4852 outliers.


In [5]:
# Keep only the columns we need going forward
columns_to_keep = ["period_x", 'date_dt', 'ticker_x', 'id', 'volatility', '1mo_rets', '1mo_fwd_rets', '1mo_log_rets', 
                   '1mo_fwd_log_rets', "1yr_rets", "1yr_log_rets",
                   "accumulated_other_comprehensive_income", "total_assets", "current_assets", "assets_non-current",
                   "asset_turnover", "book_value_per_share", "capital_expenditure", "cash_and_equivalents", 
                   "cost_of_revenue", "consolidated_income", "current_ratio",
                   "debt_to_equity_ratio", "total_debt", "debt_current", "debt_non-current",  
                   "deferred_revenue", "depreciation_amortization_accretion", "deposit_liabilities", "dividend_yield", 
                   "dividends_per_basic_common_share", "earning_before_interest_taxes_ebit", 
                   "earnings_before_interest_taxes_depreciation_amortization_ebitda", "ebitda_margin",
                   "earnings_before_tax", "earnings_per_basic_share", "earnings_per_diluted_share", 
                   "enterprise_value", "enterprise_value_over_ebit", "enterprise_value_over_ebitda", "free_cash_flow", 
                   "free_cash_flow_per_share","gross_profit", "gross_margin", 
                   "goodwill_and_intangible_assets", "interest_expense", "invested_capital", "inventory", "investments",
                   "investments_current", "investments_non-current", "total_liabilities", "current_liabilities", 
                   "liabilities_non-current", "net_cash_flow_/_change_in_cash_cash_equivalents", 
                   "net_cash_flow_-_business_acquisitions_and_disposals", "issuance_purchase_of_equity_shares",
                   "issuance_repayment_of_debt_securities_", "payment_of_dividends_other_cash_distributions___", 
                   "net_cash_flow_from_financing", "net_cash_flow_from_investing",
                   "net_cash_flow_-_investment_acquisitions_and_disposals", "net_cash_flow_from_operations", 
                   "effect_of_exchange_rate_changes_on_cash_", "net_income", "net_income_common_stock", 
                   "net_loss_income_from_discontinued_operations", 
                   "net_income_to_non-controlling_interests", "profit_margin", "operating_expenses", "operating_income", 
                   "trade_and_non-trade_payables", "payout_ratio", "price_to_book_value", "price_earnings_damodaran_method",
                   "price_to_earnings_ratio", "property_plant_equipment_net", "preferred_dividends_income_statement_impact",
                   "price_sales_damodaran_method", "price_to_sales_ratio", 
                   "trade_and_non-trade_receivables", "accumulated_retained_earnings_deficit", "revenues", 
                   "research_and_development_expense", "share_based_compensation", 
                   "selling_general_and_administrative_expense", "weighted_average_shares", "weighted_average_shares_diluted", "sales_per_share", 
                   "tangible_asset_value", "tax_assets", "income_tax_expense", "tax_liabilities", 
                   "tangible_assets_book_value_per_share", "working_capital", "capex_std","earnings_std", 
                   "gdp_change","unemp_ma_diff","confidence","MICH","bus_credit_change","con_credit_change",
                   "pce_change","Adj Close","vix_change","inverted","buyback_yield", "total_yield", "leverage", 
                   "roe", "roa", "debt_to_assets", "tp", "tp_last_yr", "intangibles_to_assets", "rnd_to_revenue", 
                   "market_capitalization"]

equity_df = equity_df[columns_to_keep]

# rename some columns
equity_df.columns = ["period", 'date_dt', 'ticker', 'id', 'volatility', '1mo_rets', '1mo_fwd_rets', '1mo_log_rets', 
                     '1mo_fwd_log_rets', "1yr_rets", "1yr_log_rets",
                   "other_comprehensive_income", "total_assets", "current_assets", "non_current_assets",
                   "asset_turnover", "book_value_per_share", "capital_expenditure", "cash_and_equivalents", 
                   "cost_of_revenue", "consolidated_income", "current_ratio",
                   "debt_to_equity", "total_debt", "debt_current", "debt_non-current",  
                   "deferred_revenue", "depreciation_amortization_accretion", "deposit_liabilities", "dividend_yield", 
                   "dividends_per_basic_common_share", "ebit", 
                   "earnings_before_interest_taxes_depreciation_amortization_ebitda", "ebitda_margin",
                   "earnings_before_tax", "earnings_per_basic_share", "earnings_per_diluted_share", 
                   "ev", "ev_to_ebit", "ev_to_ebitda", "free_cash_flow", 
                   "fcf_per_share","gross_profit", "gross_margin", 
                   "goodwill_and_intangible_assets", "interest_expense", "invested_capital", "inventory", "investments",
                   "investments_current", "investments_non-current", "total_liabilities", "current_liabilities", 
                   "liabilities_non-current", "net_cash_flow_/_change_in_cash_cash_equivalents", 
                   "business_acquisitions_and_disposals", "issuance_purchase_of_equity_shares",
                   "issuance_repayment_of_debt_securities_", "payment_of_dividends_other_cash_distributions", 
                   "net_cash_flow_from_financing", "net_cash_flow_from_investing",
                   "net_cash_flow_-_investment_acquisitions_and_disposals", "net_cash_flow_from_operations", 
                   "effect_of_exchange_rate_changes_on_cash_", "net_income", "net_income_common_stock", 
                   "netincome_from_discontinued_operations", 
                   "net_income_to_non-controlling_interests", "profit_margin", "operating_expenses", "operating_income", 
                   "trade_and_non-trade_payables", "payout_ratio", "price_to_book_value", "price_earnings_damodaran_method",
                   "price_to_earnings_ratio", "property_plant_equipment_net", "preferred_dividends_income_statement_impact",
                   "price_sales_damodaran_method", "price_to_sales_ratio", 
                   "trade_and_non-trade_receivables", "accumulated_retained_earnings_deficit", "revenue", 
                   "research_and_development_expense", "share_based_compensation", 
                   "sga_expense", "weighted_average_shares", "weighted_average_shares_diluted", "sales_per_share", 
                   "tangible_asset_value", "tax_assets", "income_tax_expense", "tax_liabilities",
                   "tangible_assets_book_value_per_share", "working_capital", "capex_std","earnings_std",
                   "gdp_change","unemp_ma_diff","confidence","MICH","bus_credit_change","con_credit_change",
                   "pce_change","vix_close","vix_change","inverted","buyback_yield", "total_yield", "leverage", 
                   "roe", "roa", "debt_to_assets", "tp", "tp_last_yr", "intangibles_to_assets", "rnd_to_revenue", 
                    "market_cap"]

In [6]:
### handling feature anomalies
from sklearn.ensemble import IsolationForest

# instantiate classifier and get predictions to locate outliers
clf = IsolationForest(n_estimators=500, n_jobs=-1, random_state=10)
preds = clf.fit_predict(equity_df.drop(["period", "date_dt", "ticker", "id"], axis=1))
equity_df["outlier"] = preds
n_outliers = equity_df['outlier'].value_counts()[-1]
clean_equity_data = equity_df.loc[equity_df["outlier"] == 1]

print(f"Percent outliers: {n_outliers/equity_df.shape[0]*100:.2f}%")

Percent outliers: 2.41%


In [7]:
# clean_equity_data.drop(["outlier"], axis=1).dropna().to_csv("C:/Users/lbianculli/dev/us_equities/us_data_final.csv")
clean_equity_data.drop(["outlier"], axis=1).dropna().to_csv("C:/Users/lbianculli/dev/us_equities/python_processes/us_data_final2.csv")

clean_equity_data.shape

(36964, 119)

### We handled the bad data, added additional metrics, and removed outliers. Next we will move onto premodeling
